In [148]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import xlrd
import tkinter as tk
from tkinter import filedialog
import math

In [149]:
# Intialize GUI interface to select file
root = tk.Tk()
root.winfo_toplevel().title("Select csv files")
root.withdraw()
# Grab Protocol File. This holds information about each trial
print("Please select Protocol File")
prot_path = filedialog.askopenfilename()
prot_path

Please select Protocol File


'V:/R01 - W4K/2_Shaker project/Devices Evaluated/Copy of W4K - Shaker Table Pilot Protocol Testing_v6.xlsx'

In [150]:
# Grab other information about file:
device = int(input("If you wish to process a Garmin trial press 0. Apple press 1: "))
trial_number = int(input("Which trial would you like to process? ")) - 1

If you wish to process a Garmin trial press 0. Apple press 1: 1
Which trial would you like to process? 1


In [ ]:
# This block creates a function that reads in the protocol sheet
def protocol_reader(path, device) :
    # Since the the file is an excel file with multiple worksheets read it in as an excel object
    all_prot = pd.ExcelFile(path)
    # Read in the first sheet as a  pandas Dataframe
    g_prot = all_prot.parse(sheet_name = all_prot.sheet_names[device])
    # Clean up the data frame. The first column is empty so drop it
    g_prot.drop(columns=g_prot.columns[[0,-1]], inplace=True)
    # The header of the data frame is empty. Replace with the first row.
    g_prot.columns = g_prot.iloc[0]
    g_prot = g_prot.iloc[1:]
    # Drop Rows with NaN values
    g_prot.dropna(axis=0, how="any", inplace=True)
    
    return g_prot

In [ ]:
garmin_prot = protocol_reader(prot_path, device)

In [ ]:
garmin_prot

In [151]:
trial = {}
proxy_count = 1
acti_count = 1
for i in range(7):
    # Grab files of trial:
    if proxy_count <= 2 :
        trial["proxy" + str(proxy_count)] = filedialog.askopenfilename()
        proxy_count += 1
    else :
        trial["acti" + str(acti_count)] = filedialog.askopenfilename()
        acti_count += 1
    
trial 

{'proxy1': 'V:/R01 - W4K/2_Shaker project/Devices Evaluated/Trial Data/Raw data/Apple/3.23.22/JKJ23D9D7J/3.23.2022_file_export_2222.csv',
 'proxy2': 'V:/R01 - W4K/2_Shaker project/Devices Evaluated/Trial Data/Raw data/Apple/3.23.22/FXWLJ49CT6/3.23.2022_file_export_4444.csv',
 'acti1': 'V:/R01 - W4K/2_Shaker project/Devices Evaluated/Trial Data/Raw data/Apple/3.23.22/Downloaded 3.23.22 GT3X Rd 1&2/csv/44RAW.csv',
 'acti2': 'V:/R01 - W4K/2_Shaker project/Devices Evaluated/Trial Data/Raw data/Apple/3.23.22/Downloaded 3.23.22 GT3X Rd 1&2/csv/97RAW.csv',
 'acti3': 'V:/R01 - W4K/2_Shaker project/Devices Evaluated/Trial Data/Raw data/Apple/3.23.22/Downloaded 3.23.22 GT3X Rd 1&2/csv/37RAW.csv',
 'acti4': 'V:/R01 - W4K/2_Shaker project/Devices Evaluated/Trial Data/Raw data/Apple/3.23.22/Downloaded 3.23.22 GT3X Rd 1&2/csv/36RAW.csv',
 'acti5': 'V:/R01 - W4K/2_Shaker project/Devices Evaluated/Trial Data/Raw data/Apple/3.23.22/Downloaded 3.23.22 GT3X Rd 1&2/csv/113RAW.csv'}

In [155]:
# Need to extract meta info about trial (trial number, round number, device ids)
# Returns a tuple of:
# Trial Number, Round Number, Dictionary of Actigraph IDs, Dictionary of Proxy IDs
def meta_extract(data, trial, prox_num, acti_num):
    # initailize data structures to hold device ids
    proxy_id = {}
    acti_id = {}
    # Extract trial and round from dataframe
    trial_num = data.iloc[trial, 5]
    round_num = data.iloc[trial, 6]
    # Hard code the indices where the ids start for each type of device
    acti_index = 7
    prox_index = 17
    # Collect all proxy ids
    for i in range(prox_num):
        proxy_id["proxy" + str(i+1)] = data.iloc[trial, prox_index]
        prox_index += 3
    # Collect all actigraph ids
    for i in range(acti_num):
        acti_id["acti" + str(i+1)] = data.iloc[trial, acti_index]
        acti_index +=2
        
    return trial_num, round_num, acti_id, proxy_id
    

In [156]:
trial_meta = meta_extract(garmin_prot, trial_number, 2, 5)

In [157]:
trial_meta

(1,
 1,
 {'acti1': 44, 'acti2': 97, 'acti3': 37, 'acti4': 36, 'acti5': 113},
 {'proxy1': 'JKJ23D9D7J', 'proxy2': 'FXWLJ49CT6'})

In [158]:
# This function  takes as input the protocol sheet, a trial number and device (actigraph or proxy)
# This function returns a dictonary of time trial start times

def trial_times(data, trial_num):
    acti_time = {}
    proxy_time = {}
    # This is the index where the first actigraph trial start time is listed
    acti_index = -18
    prox_index = acti_index + 1
    # Grab date of the trial this is listed in column 3
    date = data.iloc[trial_num, 3]
    
    for i in range(7):
        # This gets the start time from the protocol sheet
        acti_start = data.iloc[trial_num, acti_index]
        prox_start = data.iloc[trial_num, prox_index]
        # This adds the date and time to the dictionaries
        acti_time['setting_' + str(7-i)] = datetime.combine(date, acti_start) 
        proxy_time['setting_' + str(7-i)] = datetime.combine(date, prox_start)
        
        # Increment both the indices
        acti_index += 2
        prox_index += 2
        
    return acti_time, proxy_time
    

In [159]:
# A tuple of dictonaries 
# actigraph startimes in position 0
# Proxy statimes in position 1
start_times = trial_times(garmin_prot, trial_number)

In [160]:
start_times[0]

{'setting_7': datetime.datetime(2022, 3, 23, 9, 23, 15),
 'setting_6': datetime.datetime(2022, 3, 23, 9, 25, 15),
 'setting_5': datetime.datetime(2022, 3, 23, 9, 27, 15),
 'setting_4': datetime.datetime(2022, 3, 23, 9, 29, 15),
 'setting_3': datetime.datetime(2022, 3, 23, 9, 31, 15),
 'setting_2': datetime.datetime(2022, 3, 23, 9, 33, 15),
 'setting_1': datetime.datetime(2022, 3, 23, 9, 35, 15)}

In [161]:
start_times[1]

{'setting_7': datetime.datetime(2022, 3, 23, 9, 23, 15),
 'setting_6': datetime.datetime(2022, 3, 23, 9, 25, 15),
 'setting_5': datetime.datetime(2022, 3, 23, 9, 27, 15),
 'setting_4': datetime.datetime(2022, 3, 23, 9, 29, 15),
 'setting_3': datetime.datetime(2022, 3, 23, 9, 31, 15),
 'setting_2': datetime.datetime(2022, 3, 23, 9, 33, 15),
 'setting_1': datetime.datetime(2022, 3, 23, 9, 35, 15)}

In [162]:
# This block will define a function that takes as input a dictionary of file paths
# This function returns a dictionary of dataframes
def device_reader(files, device) :
    prox_count = 1
    acti_count = 1
    data = {}
    for path in files :
        # Reads in garmin files
        if path[:5] == "proxy" and device == 0:
            data["proxy" + str(prox_count)] = pd.read_csv(files[path])
            
            prox_count += 1
        # Reads in apple devices
        elif path[:5] == "proxy" and device == 1:
            # Need to check the delimeter of the file. If the delimiter is True it's comma delimited
            # intitally assume comma
            delim = True
            # open file and peek at first line
            check = open(files[path])
            line = check.readline()
            # determine if it is pipe or comma delimited
            for character in line :
                if character == '|' :
                    delim = False
                    break
                elif character == ',' :
                    break
            check.close()
            # Now read in the file based upon delimiter
            if delim :
                data["proxy" + str(prox_count)] = pd.read_csv(files[path])
            else :
                data["proxy" + str(prox_count)] = pd.read_csv(files[path], delimiter='|')
            prox_count += 1  
            
        else:
            data["acti" + str(acti_count)] = pd.read_csv(files[path], skiprows=10)
            acti_count += 1
    return data
            
        

In [163]:
test = device_reader(trial, device)

C:\Users\Nick\AppData\Local\Temp/ipykernel_9160/3245562449.py:1: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  test = device_reader(trial, device)


In [164]:
test.keys()

dict_keys(['proxy1', 'proxy2', 'acti1', 'acti2', 'acti3', 'acti4', 'acti5'])

In [165]:
test['proxy1']

,loggingTime(txt),accelerometerTimestamp_sinceReboot(s),accelerometerAccelerationX(G),accelerometerAccelerationY(G),accelerometerAccelerationZ(G),motionTimestamp_sinceReboot(s),motionYaw(rad),motionRoll(rad),motionPitch(rad),motionRotationRateX(rad/s),...,motionQuaternionZ(R),motionQuaternionW(R),motionGravityX(G),motionGravityY(G),motionGravityZ(G),motionMagneticFieldX(µT),motionMagneticFieldY(µT),motionMagneticFieldZ(µT),motionHeading(°),motionMagneticFieldCalibrationAccuracy(Z)
0,2022-03-23T09:05:11.430-04:00,6242.257234,0.203369,-0.780426,-0.577194,6242.252244,-0.160815,0.359200,0.933424,0.139289,...,0.009541,0.882305,0.209188,-0.803662,-0.557107,0.0,0.0,0.0,-1.0,-1.0
1,2022-03-23T09:05:11.432-04:00,6242.267214,0.241135,-0.776291,-0.445328,6242.262224,-0.159639,0.356388,0.933175,-0.255362,...,0.009397,0.882526,0.207690,-0.803514,-0.557880,0.0,0.0,0.0,-1.0,-1.0
2,2022-03-23T09:05:11.442-04:00,6242.277194,0.179214,-0.814697,-0.442612,6242.272204,-0.157321,0.355331,0.930539,-0.231860,...,0.009927,0.883144,0.207837,-0.801942,-0.560083,0.0,0.0,0.0,-1.0,-1.0
3,2022-03-23T09:05:11.452-04:00,6242.287173,0.167480,-0.791840,-0.629959,6242.282183,-0.157062,0.356165,0.930171,0.125659,...,0.010195,0.883170,0.208406,-0.801722,-0.560186,0.0,0.0,0.0,-1.0,-1.0
4,2022-03-23T09:05:11.462-04:00,6242.297153,0.182632,-0.777313,-0.657547,6242.292163,-0.157957,0.357162,0.931484,0.120424,...,0.010152,0.882832,0.208597,-0.802506,-0.558991,0.0,0.0,0.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180208,2022-03-23T10:03:24.548-04:00,7461.501714,0.120117,-0.290482,-0.971100,7461.491735,-0.217000,0.104327,0.301256,-0.100017,...,-0.099138,0.982374,0.099448,-0.296720,-0.949772,0.0,0.0,0.0,-1.0,-1.0
180209,2022-03-23T10:03:24.558-04:00,7461.511693,0.115402,-0.299652,-0.985855,7461.501714,-0.217100,0.103472,0.300259,-0.092207,...,-0.099286,0.982455,0.098666,-0.295768,-0.950151,0.0,0.0,0.0,-1.0,-1.0
180210,2022-03-23T10:03:24.568-04:00,7461.521672,0.105194,-0.318054,-0.997818,7461.511693,-0.217020,0.101884,0.299365,-0.100506,...,-0.099398,0.982550,0.097185,-0.294914,-0.950569,0.0,0.0,0.0,-1.0,-1.0
180211,2022-03-23T10:03:24.578-04:00,7461.531651,0.102219,-0.317978,-1.006073,7461.521672,-0.216712,0.100368,0.298032,-0.170228,...,-0.099407,0.982685,0.095782,-0.293639,-0.951106,0.0,0.0,0.0,-1.0,-1.0


In [166]:
# This function converts the garmin, apple, and actigraph timestamps into python datetimes. This allows 
# comparison of times
# To inputs data and device
# data is a dictionary of dataframes, corresponds to data from 7 devices (2 proxy and 5 actigraph)
# device specifies proxy (apple watch or garmin)
def simple(aTime) :
    aTime.microsecond = 0
    
def proxy_time_convert(data, device):
    for table in data :
        if table[:5] == "proxy" :
            if device == 0:
                # Converts garmin timestamp to python datetime
                data[table]["Time"] = data[table]['Time'].apply(lambda x: str(xlrd.xldate_as_datetime(x, 0)))
                data[table]["Time"] = pd.to_datetime(data[table]["Time"])
                data[table]["Time"] = data[table]["Time"].apply(lambda x: x.replace(microsecond=0) )
                
            else :
                # Rename time and acceleration columns
                data[table].rename(columns={'loggingTime(txt)': 'Time', 'accelerometerTimestamp_sinceReboot(s)': 'Reading #',
                                            'accelerometerAccelerationX(G)': 'X', 'accelerometerAccelerationY(G)': 'Y',
                                            "accelerometerAccelerationZ(G)": 'Z'}, inplace = True)
                # Convert timestamp into datetime
                data[table]["Time"] = data[table]["Time"].apply(lambda x: datetime.fromisoformat(x[:-6]))
                data[table]["Time"] = pd.to_datetime(data[table]["Time"])
                data[table]["Reading #"] = data[table]["Reading #"].apply(lambda x: np.nan)
                data[table] = data[table].iloc[:,:5]
                
        else :
            # Converts actigraph timestamp into python datetime
            data[table]['Timestamp'] = data[table]["Timestamp"].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M:%S.%f'))
            data[table] = data[table].rename(columns = {'Timestamp':'Time'})
    

In [167]:
proxy_time_convert(test, device)

In [168]:
test["proxy2"]

,Time,Reading #,X,Y,Z
0,2022-03-23 09:16:48.148,NaN,-0.021149,-0.188934,-1.010788
1,2022-03-23 09:16:48.156,NaN,-0.123932,-0.263977,-0.836273
2,2022-03-23 09:16:48.159,NaN,-0.102829,-0.261826,-0.860687
3,2022-03-23 09:16:48.168,NaN,-0.042114,-0.250809,-1.001694
4,2022-03-23 09:16:48.178,NaN,-0.023056,-0.249908,-1.031876
...,...,...,...,...,...
143441,2022-03-23 10:03:58.617,NaN,0.073959,-0.273682,-0.991898
143442,2022-03-23 10:03:58.627,NaN,0.074951,-0.282990,-0.995193
143443,2022-03-23 10:03:58.637,NaN,0.078720,-0.296616,-0.999649
143444,2022-03-23 10:03:58.647,NaN,0.087936,-0.300934,-0.997330


In [169]:
# This function reads through each data frame and grabs the minutes needed.
# To inputs devices which is a dictionary of dataframes and
# Times which is a tuple of dictionaries of the start times
# This function returns a dictionary of dataframes that have been filtered to only contain data during the trial
def minute_extractor(devices, times, meta_data) :
    # iterate through each device
    filtered = {}
    speeds = ["0.6 Hz", "1.0 Hz", "1.5 Hz", "1.9 Hz", "2.4 Hz", "2.8 Hz", "3.2 Hz"]
    
    for data in devices:
        # used to keep track of location in dataframe
        index = 0
        # Store the  dataframes in a list.
        readings = None
        # Since proxy and acti can have different start times need to check which device
        trials = None
        if data[:5] == "proxy":
            # Get dictionary of start times
            trials = times[1]
            # Get Device ID
            dev_id = meta_data[3][data]
        else :
            trials = times[0]
            dev_id = meta_data[2][data]
        # intialize speed of trial
        speed_index = 6
        # Iterates the start times
        for trial in trials:
            start = trials[trial]
            # Each trial is 2 minutes long. To get the end time add 2 minutes.
            end = start + timedelta(minutes=2)
            # Filter the device data to be between the start and end time
            temp = devices[data].loc[(devices[data]['Time'] >= start) & (devices[data]["Time"] < end)]
            # Next I will insert the speed into the dataframe
            temp.insert(0, "Speed", speeds[speed_index], True)
            speed_index -= 1
            # Insert include column
            temp.insert(2, "Include", 0, True)
            temp.loc[ (temp.Time >= (start + timedelta(seconds=30))) & (temp.Time < (end - timedelta(seconds=30))), "Include" ] = 1
            # Add filtered data to new dataframe
            if readings is None :
                readings = temp
            else :
                readings = pd.concat([readings, temp], axis=0)
                
        # Insert rest of trial meta data into dataframe
        readings.insert(0, "Trial Number", meta_data[0], True )
        readings.insert(1, "Round Number", meta_data[1], True )
        readings.insert(3, "ID", dev_id, True)
        filtered[data] = readings
         
    return filtered
            
    

In [170]:
trial = minute_extractor(test, start_times, trial_meta)

C:\Users\Nick\Anaconda3\envs\watch_extract\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [171]:
trial

{'proxy1':        Trial Number  Round Number   Speed          ID                    Time  \
 57680             1             1  3.2 Hz  JKJ23D9D7J 2022-03-23 09:23:15.009   
 57681             1             1  3.2 Hz  JKJ23D9D7J 2022-03-23 09:23:15.029   
 57682             1             1  3.2 Hz  JKJ23D9D7J 2022-03-23 09:23:15.050   
 57683             1             1  3.2 Hz  JKJ23D9D7J 2022-03-23 09:23:15.070   
 57684             1             1  3.2 Hz  JKJ23D9D7J 2022-03-23 09:23:15.090   
 ...             ...           ...     ...         ...                     ...   
 99771             1             1  0.6 Hz  JKJ23D9D7J 2022-03-23 09:37:14.917   
 99772             1             1  0.6 Hz  JKJ23D9D7J 2022-03-23 09:37:14.937   
 99773             1             1  0.6 Hz  JKJ23D9D7J 2022-03-23 09:37:14.957   
 99774             1             1  0.6 Hz  JKJ23D9D7J 2022-03-23 09:37:14.977   
 99775             1             1  0.6 Hz  JKJ23D9D7J 2022-03-23 09:37:14.997   
 
    

In [172]:
# This function takes as input 2 data frames an actigraph and a proxy
# It then converts them to numpy arrays and iterates through them, comparing the times of each reading.
# It returns 1 data frame with readings from both dataframe, ordered by time.
def data_compare(actigraph, proxy):
    # Get shape of dataframes
    a_rows, a_columns = actigraph.shape
    p_rows, p_columns = proxy.shape
    max_rows = max(a_rows, p_rows)
    output = np.empty([max_rows, a_columns + p_columns], dtype="O")
    # Convert to numpy arrays
    n_acti = actigraph.to_numpy()
    n_prox = proxy.to_numpy()
    # intialize variables before loop
    acti = None
    prox = None
    p_row = 0
    # Iterate through both devices aligning data.
    for row in range(max_rows):
        out_row = []
        # Get next actigraph reading
        if row < a_rows :
            acti = n_acti[row]
        # Get proxy reading
        if p_row < p_rows :
            prox = n_prox[p_row]
        # Add actigraph to row
        for item in acti :
            out_row.append(item)
        # Check if proxy row should be added
        if acti[4] >= prox[4] and p_row < p_rows :
            # add proxy reading
            for item in prox :
                out_row.append(item)
            # Increment row in proxy
            p_row += 1
        else :
            # If no prox reading fill last 9 columns with NaNs
            for i in range(p_columns):
                out_row.append(np.nan)
        # Add row to output array
        # print(out_row)
        output[row, :] = out_row
    # Get new columns
    new_columns = ['Trial Number', 'Round Number', 'Speed', 'Actigraph ID', 'Actigraph Time','Include', 'Actigraph X', 
                   'Actigraph Y', 'Actigraph Z', 'Trial Number2', 'Round Number2', 'Speed2', 'Proxy ID', 'Proxy Time',
                  "Include2", "Reading #", "Proxy X", "Proxy Y", "Proxy Z"]
    output_df = pd.DataFrame(output, columns=new_columns)
    output_df.drop(columns=["Trial Number2", "Round Number2", "Speed2","Include2"], inplace=True)
    return output_df

In [173]:
# This function aligns every actigraph's data from the trial with every proxy(garmin/apple) from the trial
# Every actigraph device will be compared to each proxy 
# The input to this function is a dictionary of the filtered trial data.
# There are 2 outputs:
# 1 a dataframe that contains all possible actigraphs alinged with all possible proxy devices(This is later stored as a csv)
# 2 a dictionary of dataframes that contain 1 actigraph alinged with 1 device(Used late to get inner minute)
def device_aligner(devices):
    # This loop iterates through the dictionary and finds each actigraph device.
    aligned = None
    pair_count = 1
    pairs = {}
    for a_data in devices :
        # Checks if the device is an actigraph
        if a_data[:-1] == "acti" : 
            # This loop iterates through the dictionary and finds each proxy device(apple/garmin)
            for p_data in devices :
                if p_data[:-1] == "proxy":
                    print(f"Aligning: {a_data} with {p_data}")
                    if aligned is None :
                        # Data compare combines to data frames
                        aligned = data_compare(devices[a_data], devices[p_data])
                        pairs["pair" + str(pair_count)] = aligned
                    else :
                        temp = data_compare(devices[a_data], devices[p_data])
                        pairs["pair" + str(pair_count)] = temp
                        aligned = pd.concat([aligned, temp], axis=0)
                    pair_count += 1
    return aligned, pairs

In [174]:
aligned_data, device_pairs = device_aligner(trial)

Aligning: acti1 with proxy1
Aligning: acti1 with proxy2
Aligning: acti2 with proxy1
Aligning: acti2 with proxy2
Aligning: acti3 with proxy1
Aligning: acti3 with proxy2
Aligning: acti4 with proxy1
Aligning: acti4 with proxy2
Aligning: acti5 with proxy1
Aligning: acti5 with proxy2


In [175]:
aligned_data

,Trial Number,Round Number,Speed,Actigraph ID,Actigraph Time,Include,Actigraph X,Actigraph Y,Actigraph Z,Proxy ID,Proxy Time,Reading #,Proxy X,Proxy Y,Proxy Z
0,1,1,3.2 Hz,44,2022-03-23 09:23:15.000,0,-0.301,-0.285,-1.082,NaN,NaT,NaN,NaN,NaN,NaN
1,1,1,3.2 Hz,44,2022-03-23 09:23:15.010,0,-0.32,-0.234,-1.055,JKJ23D9D7J,2022-03-23 09:23:15.009,NaN,-0.028564,-0.008545,-0.991943
2,1,1,3.2 Hz,44,2022-03-23 09:23:15.020,0,-0.316,-0.184,-1.02,NaN,NaT,NaN,NaN,NaN,NaN
3,1,1,3.2 Hz,44,2022-03-23 09:23:15.030,0,-0.348,-0.121,-1.008,JKJ23D9D7J,2022-03-23 09:23:15.029,NaN,-0.0354,0.005371,-1.005859
4,1,1,3.2 Hz,44,2022-03-23 09:23:15.040,0,-0.406,-0.031,-1.023,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83995,1,1,0.6 Hz,113,2022-03-23 09:37:14.950,0,0.031,-0.02,-1.039,FXWLJ49CT6,2022-03-23 09:37:14.944,NaN,-0.0271,-0.005615,-1.001465
83996,1,1,0.6 Hz,113,2022-03-23 09:37:14.960,0,0.031,-0.02,-1.039,NaN,NaT,NaN,NaN,NaN,NaN
83997,1,1,0.6 Hz,113,2022-03-23 09:37:14.970,0,0.031,-0.02,-1.039,FXWLJ49CT6,2022-03-23 09:37:14.964,NaN,-0.029785,-0.000732,-0.989014
83998,1,1,0.6 Hz,113,2022-03-23 09:37:14.980,0,0.031,-0.02,-1.039,NaN,NaT,NaN,NaN,NaN,NaN


In [178]:
path = "C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Tests"
if device == 0:
    path = path + "\\Garmin_Trial" + str(trial_meta[1] + 1) + ".csv"
else :
    path = path + "\\Apple_Trial" + str(trial_meta[1] + 1) + ".csv"
aligned_data.to_csv(path, index=False)

I'm Here


In [179]:
device_pairs.keys()

dict_keys(['pair1', 'pair2', 'pair3', 'pair4', 'pair5', 'pair6', 'pair7', 'pair8', 'pair9', 'pair10'])

In [180]:
# This function takes as input a dataframe containing trial data for an aligned actigraph and proxy pair
# The output of this function is the inner minute data of each trial for this pair.
def inner_minute(data):
    outer_min, inner_min = data.groupby(by="Include")
    return inner_min[1]
    

In [184]:
def root_mean(dataframe, device):
    # Get shape of dataframe
    # print(f"Data Frame shape {dataframe.shape}")
    rows, columns = dataframe.shape
    # The convert to a numpy array
    data = dataframe.to_numpy()
    # Initialize output
    # Check which device the proxy is. Then divide the number of rows by the sample rate of the actigraph.
    # This should result in the ouput data frame having 420 rows, which corresponds to 7 minutes worth of readings
    # with a sample every second
    if device == 0 :
        out_rows = rows // 30
    else :
        out_rows = rows // 100
        
    # print(f"Out rows {out_rows}")
    data_out = np.zeros([out_rows, columns + 3], dtype="O")
    # out_row is the being written to in data_out
    out_row = 0
    # row is the row being read from in data
    row = 0
    prox_id = data[row,9]
    if np.isnan(prox_id):
        prox_id=data[row+1,9]
    while out_row < out_rows:
        #start represents the first row in data that begins a new second
        start = data[row,:]
        # Gets the time from row
        start_sec = start[4]
        # Calculates the next second
        end_sec = start_sec + timedelta(seconds=1)
        # intialize avg_sum,square_sum, and count which are used to calculate Avg and RMS
        # Each value in the lists correspond to an accel value Acti xyz and Proxy xyz
        avg = [0,0,0,0,0,0]
        squ = [0,0,0,0,0,0]
        acti_count = 0
        prox_count = 0
        # Iterates through seconds worth of data
        while row < rows and data[row, 4] < end_sec:
            # Add each reading to corresponding value in avg_sum
            avg[0] += data[row, 6]
            avg[1] += data[row, 7]
            avg[2] += data[row, 8]
            # Need to check if the proxy reading is a NaN value
            if not np.isnan(data[row, 12]):
                avg[3] += data[row, 12]
                avg[4] += data[row, 13]
                avg[5] += data[row, 14]
                
            # Add each reading to corresponding value in squ_sum
            squ[0] += data[row, 6]**2
            squ[1] += data[row, 7]**2
            squ[2] += data[row, 8]**2
            if not np.isnan(data[row, 12]):             
                squ[3] += data[row, 12]**2
                squ[4] += data[row, 13]**2
                squ[5] += data[row, 14]**2
                prox_count += 1
            
            acti_count += 1
            row += 1
        # Now to calculate average and rms
        for i in range(6):
            # Since the sampling rate of actigraph is higher than proxy need to use two different counts
            if i < 3:
                avg[i] = avg[i] / acti_count
                squ[i] = squ[i] / acti_count
            else :
                avg[i] = avg[i] / prox_count
                squ[i] = squ[i] / prox_count                
            
            squ[i] = math.sqrt(squ[i])
        # Add the needed values to the output row
        # Add trial # round # speed actigraph id and Time
        data_out[out_row, :5] = start[:5]
        # Add actigraph xyz mean
        data_out[out_row, 5:8] = avg[:3]
        # add actigraph xyz RMS
        data_out[out_row, 8:11] = squ[:3]
        # Add Proxy ID
        data_out[out_row, 11] = prox_id
        # Add Proxy xyz mean
        data_out[out_row, 12:15] = avg[3:]
        # add proxy xyz rms
        data_out[out_row, 15:] = squ[3:]
        
        out_row += 1
    headers = ['Trial Number', 'Round Number', 'Speed', 'Actigraph ID', 'Time', 'Actigraph Mean X', 'Actigraph Mean Y',
              'Actigraph Mean Z', 'Actigraph RMS X', 'Actigraph RMS Y', 'Actigraph RMS Z', 'Proxy ID', 'Proxy Mean X',
              'Proxy Mean Y', 'Proxy Mean Z', 'Proxy RMS X', 'Proxy RMS Y', 'Proxy RMS Z']
    df_out = pd.DataFrame(data_out, columns = headers)
    return df_out
        
    

In [182]:
# This function pulls the inner minute of each device pair, calculates the RMS and average values at a second level
# It takes as input a dictionary of dataframes where each dataframe corresponds to 1 actigraph and proxy aligned trial data
# and the start time of each trial
# It outputs a single data frame that includes each actigraph proxy pair, where the data is aggregated to the second level 
# And the RMS is calculated for each second.
def aggregate(pairs, device):
    output_df = None
    # Iterate through pairs dictionary
    for pair in pairs:
        # First the inner minute of each trial needs to be extracted from each data frame:
        inner_data = inner_minute(pairs[pair])
        # Calculate the RMS and average of acceleration data
        rms = root_mean(inner_data, device)
        if output_df is None:
            output_df = rms
        else:
            output_df = pd.concat([output_df, rms], axis=0)
        
    return output_df

In [185]:
rms_data = aggregate(device_pairs,device)
rms_data

,Trial Number,Round Number,Speed,Actigraph ID,Time,Actigraph Mean X,Actigraph Mean Y,Actigraph Mean Z,Actigraph RMS X,Actigraph RMS Y,Actigraph RMS Z,Proxy ID,Proxy Mean X,Proxy Mean Y,Proxy Mean Z,Proxy RMS X,Proxy RMS Y,Proxy RMS Z
0,1,1,3.2 Hz,44,2022-03-23 09:23:45,0.08527,0.02374,-1.01724,0.351705,0.386596,1.018146,JKJ23D9D7J,-0.077383,0.003716,-0.992876,0.405211,0.354133,0.994725
1,1,1,3.2 Hz,44,2022-03-23 09:23:46,0.00733,-0.02669,-1.01716,0.339555,0.366823,1.017973,JKJ23D9D7J,0.015073,-0.012915,-0.99749,0.38792,0.343991,0.998914
2,1,1,3.2 Hz,44,2022-03-23 09:23:47,0.08201,0.02631,-1.01685,0.349143,0.381584,1.017732,JKJ23D9D7J,-0.074038,0.009888,-0.992168,0.401475,0.3485,0.994019
3,1,1,3.2 Hz,44,2022-03-23 09:23:48,0.00762,-0.02824,-1.01728,0.344741,0.372847,1.018172,JKJ23D9D7J,0.011895,-0.019692,-0.996577,0.386935,0.346242,0.998091
4,1,1,3.2 Hz,44,2022-03-23 09:23:49,0.08436,0.02376,-1.0182,0.350132,0.383272,1.019072,JKJ23D9D7J,-0.072827,0.013745,-0.992905,0.405335,0.35166,0.994656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,1,1,0.6 Hz,113,2022-03-23 09:36:40,0.031,-0.02,-1.039,0.031,0.02,1.039,FXWLJ49CT6,-0.019604,0.002026,-0.995728,0.02133,0.005603,0.995739
416,1,1,0.6 Hz,113,2022-03-23 09:36:41,0.031,-0.02,-1.039,0.031,0.02,1.039,FXWLJ49CT6,-0.016133,-0.009575,-0.995942,0.020285,0.014993,0.996009
417,1,1,0.6 Hz,113,2022-03-23 09:36:42,0.031,-0.02,-1.039,0.031,0.02,1.039,FXWLJ49CT6,-0.019187,0.001632,-0.996008,0.02076,0.005495,0.996012
418,1,1,0.6 Hz,113,2022-03-23 09:36:43,0.031,-0.02,-1.039,0.031,0.02,1.039,FXWLJ49CT6,-0.016431,-0.009902,-0.996133,0.021272,0.015586,0.996184


In [186]:
path = "C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Tests"
if device == 0:
    path = path + "\\Garmin_RMS_Trial" + str(trial_meta[1]) + ".csv"
else :
    path = path + "\\Apple_RMS_Trial" + str(trial_meta[1]) + ".csv"
rms_data.to_csv(path, index=False)